In [1]:
import os

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from matplotlib import colors

from scipy import stats

import xarray as xr
import geopandas as gpd
import xesmf as xe
import rioxarray as rio
#import geoviews as gv

#import geoviews.feature as gf

#gv.extension("matplotlib")
#gv.output( size = 600 )

In [2]:
# Datos

# Códigos nacionales y subnacionales
ix  = "ISO_A3"

# Carpetas
path_catalog = "../../Bases_de_datos/Data_catalog.csv"
data_d = "../results/"
iso = "../../Bases_de_datos/Country_ISO_code.csv"

# Relación de códigos Banco Mundial y Natural Earth (ISO 3166-2)
code_path = "../../Bases_de_datos/Subnational_Code_WB_ISO.csv"

# Catálogo de datos
df_c = pd.read_csv(path_catalog)


df_iso = pd.read_csv(iso).rename(columns = {"alpha-3": ix}).set_index(ix)

df_c.head()

,Dataset,ID,Origin,Countries,Granularity,Timestep,Annex,Website,Filename,Path
0,Exposure to sea level rise,pone.0118571.s004,Neumann et al.,All,National,Periods,NaN,https://doi.org/10.1371/journal.pone.0118571,pone.0118571.s004.csv,Bases_de_datos/Sea_level_rise/
1,Air Emissions Accounts,DF_AEA,OECD,All,National,"OECD: Quarterly, Non-OECD: Yearly",NaN,https://data-explorer.oecd.org,"OECD.SDD.NAD.SEEA,DSD_AEA@DF_AEA,1.0+all.csv",Bases_de_datos/OECD_Air_and_Climate/
2,Air emissions - Air pollutants Inventories,DF_AIR_EMISSIONS,OECD,All,National,Yearly,Air Emissions Correspondance Table.xlsx,https://data-explorer.oecd.org,"OECD.ENV.EPI,DSD_AIR_EMISSIONS@DF_AIR_EMISSION...",Bases_de_datos/OECD_Air_and_Climate/
3,Air emissions - Greenhouse gas emissions Inven...,DF_AIR_GHG,OECD,All,National,Yearly,NaN,https://data-explorer.oecd.org,"OECD.ENV.EPI,DSD_AIR_GHG@DF_AIR_GHG,1.0+all.csv",Bases_de_datos/OECD_Air_and_Climate/
4,Air transport CO2 emissions (experimental),DF_AIR_TRANSPORT,OECD,All,National,Monthly,NaN,https://data-explorer.oecd.org,"OECD.SDD.NAD.SEEA,DSD_AIR_TRANSPORT@DF_AIR_TRA...",Bases_de_datos/OECD_Air_and_Climate/


In [3]:
vars = ["HDI", "GDI", "Total Debt [% of GNI]",
    #"R&D expenditure [% of GDP]",
    "Scientific and Technical Journal Articles",
    "Gini Coefficient"]
var_i = [v + "_index" for v in vars]

id = "GDI_HDI"

# Cargamos el archivo
df = pd.read_csv( "../../" + df_c.loc[df_c["ID"]==id, "Path"].iloc[0]
    + df_c.loc[df_c["ID"]==id, "Filename" ].iloc[0],
    index_col = ix ).drop(columns = ["Source"])

print( df[ df[vars[1]].isnull() ].shape )
df.sort_values(vars[0])

(49, 4)


,Name,HDI,GDI,SVN_A3
ISO_A3,,,,
CXR,Christmas Island,0.310,NaN,AUS
SOM,Somalia,0.380,0.769,SOM
SSD,South Sudan,0.381,0.843,SSD
CAF,Central African Republic,0.387,0.810,CAF
NER,Niger,0.394,0.826,NER
...,...,...,...,...
ISL,Iceland,0.959,0.975,ISL
NOR,Norway,0.966,0.986,NOR
CHE,Switzerland,0.967,0.971,CHE


In [4]:
id = "DEM"
indicator = "DT.TDS.DECT.GN.ZS"
i = 2

# Cargamos el archivo
df_i = pd.read_csv( "../../" + df_c.loc[df_c["ID"]==id, "Path"].iloc[0]
    + df_c.loc[df_c["ID"]==id, "Filename" ].iloc[0]
    ).rename( columns = {"COUNTRY_ID": ix} )
df = df.reset_index().set_index("SVN_A3")
df[vars[i]] = df_i[ df_i["INDICATOR_ID"] == indicator ].sort_values(
    [ix, "YEAR"] ).drop_duplicates(ix, keep = "last").set_index(
    ix).rename( columns = {"VALUE": vars[i]} )[ vars[i] ]

#id = "DEBT"
#i = 2

# Cargamos el archivo
#df_i = pd.read_csv( "../../" + df_c.loc[df_c["ID"]==id, "Path"].iloc[0]
#    + df_c.loc[df_c["ID"]==id, "Filename" ].iloc[0],
#    ).rename( columns = {"Country Code": ix} ).set_index(ix)
#df = df.reset_index().set_index("SVN_A3")
#df[vars[i]] = df_i["2022 [YR2022]"]

print( df[ df[vars[i]].isnull() ].shape )
df.sort_values(vars[i])

(120, 5)


,ISO_A3,Name,HDI,GDI,Total Debt [% of GNI]
SVN_A3,,,,,
SYR,SYR,Syrian Arab Republic,0.557,0.805,0.01170
IRN,IRN,Iran (Islamic Republic of),0.780,0.880,0.09557
DZA,DZA,Algeria,0.745,0.881,0.11969
HTI,HTI,Haiti,0.552,0.929,0.12100
AFG,AFG,Afghanistan,0.462,0.622,0.17753
...,...,...,...,...,...
ARE,ARE,United Arab Emirates,0.937,0.986,NaN
GBR,GBR,United Kingdom,0.940,0.976,NaN
USA,USA,United States,0.927,1.005,NaN


In [5]:
#id = "SCN-SDG"
#indicator = "EXPGDP.TOT"
#indicator = "RESDEN.INHAB.TFTE"
#i = 3

# Cargamos el archivo
#df_i = pd.read_csv( "../../" + df_c.loc[df_c["ID"]==id, "Path"].iloc[0]
#    + df_c.loc[df_c["ID"]==id, "Filename" ].iloc[0]
#    ).rename( columns = {"COUNTRY_ID": ix} )
#df = df.reset_index().set_index("SVN_A3")
#df[vars[i]] = df_i[ df_i["INDICATOR_ID"] == indicator ].sort_values(
#    [ix, "YEAR"] ).drop_duplicates(ix, keep = "last").set_index(
#    ix).rename( columns = {"VALUE": vars[i]} )[ vars[i] ]

id = "IP.JRN.ARTC.SC"
i = 3

# Cargamos el archivo
df_i = pd.read_csv( "../../" + df_c.loc[df_c["ID"]==id, "Path"].iloc[0]
    + df_c.loc[df_c["ID"]==id, "Filename" ].iloc[0], skiprows = 3
    ).rename( columns = {"Country Code": ix} )
df = df.reset_index().set_index("SVN_A3")
#df = df.reset_index().set_index(ix)
df[vars[i]] = df_i.set_index(ix).iloc[ :, 4:-1 ].ffill(axis = 1).iloc[:, -1]

print( df[ df[vars[i]].isnull() ].shape )
df.sort_values(vars[i])

(3, 6)


,ISO_A3,Name,HDI,GDI,Total Debt [% of GNI],Scientific and Technical Journal Articles
SVN_A3,,,,,,
TUV,TUV,Tuvalu,0.653,0.975,NaN,0.39
NRU,NRU,Nauru,0.696,1.037,NaN,0.90
MHL,MHL,Marshall Islands,0.731,0.945,NaN,1.58
GNQ,GNQ,Equatorial Guinea,0.650,NaN,NaN,1.70
STP,STP,Sao Tome and Principe,0.613,0.907,0.49014,2.09
...,...,...,...,...,...,...
USA,VIR,U.S. Virgin Islands,0.849,NaN,NaN,455855.57
CHN,CHN,China,0.788,0.962,1.90711,669744.30
TWN,TWN,Chinese Taipei,0.907,NaN,NaN,NaN


In [6]:
id = "SI.POV.GINI"
i = 4

# Cargamos el archivo
df_i = pd.read_csv( "../../" + df_c.loc[df_c["ID"]==id, "Path"].iloc[0]
    + df_c.loc[df_c["ID"]==id, "Filename" ].iloc[0][:-4] + "_extra.csv",
    skiprows = 4 ).rename( columns = {"Country Code": ix}
    ).drop(columns = ["Source"])
df = df.reset_index().set_index(ix)
df[vars[i]] = df_i.set_index(ix).iloc[ :, 4: ].ffill(axis = 1).iloc[:, -1]

print( df[ df[vars[i]].isnull() ].shape )
df.sort_values(vars[i])

(0, 7)


,SVN_A3,Name,HDI,GDI,Total Debt [% of GNI],Scientific and Technical Journal Articles,Gini Coefficient
ISO_A3,,,,,,,
FRO,DNK,Faroe Islands,0.899,NaN,NaN,15318.52,22.7
VGB,GBR,British Virgin Islands,0.823,NaN,NaN,105564.47,23.0
SVK,SVK,Slovakia,0.855,1.002,NaN,5420.07,24.1
SVN,SVN,Slovenia,0.926,0.999,NaN,3827.50,24.3
BLR,BLR,Belarus,0.801,1.003,6.46915,1352.11,24.4
...,...,...,...,...,...,...,...
BHS,BHS,Bahamas,0.820,1.007,NaN,22.73,57.5
NAM,NAM,Namibia,0.610,1.006,NaN,198.74,59.1
BHR,BHR,Bahrain,0.888,0.937,NaN,492.66,59.6


In [7]:
index_n = "Socioeconomic risk index"

df = df.reset_index().set_index(ix)

var_i = [v + "_index" for v in vars]
for i, v in enumerate(vars):
    min_v = df[v].min()
    max_v = df[v].max()
    rng_v = max_v - min_v
    df[ var_i[i] ] = 10 * ( df[v] - min_v ) / rng_v
    if v == "Gini Coefficient": pass
    else: df[ var_i[i] ] = 10 - df[ var_i[i] ]

df[index_n] = df[var_i].mean(axis = 1)
min_v = df[index_n].min()
max_v = df[index_n].max()
rng_v = max_v - min_v
df[ index_n ] = 10 * ( df[index_n] - min_v ) / rng_v

df.sort_values(index_n, ascending = False).head(30)

,SVN_A3,Name,HDI,GDI,Total Debt [% of GNI],Scientific and Technical Journal Articles,Gini Coefficient,HDI_index,GDI_index,Total Debt [% of GNI]_index,Scientific and Technical Journal Articles_index,Gini Coefficient_index,Socioeconomic risk index
ISO_A3,,,,,,,,,,,,,
YEM,YEM,Yemen,0.424,0.456,0.51552,282.60,36.70,8.564232,10.000000,9.892564,9.995786,3.473945,10.000000
ERI,ERI,Eritrea,0.493,NaN,1.08504,37.51,38.00,7.695214,NaN,9.771117,9.999446,3.796526,9.016418
CAF,CAF,Central African Republic,0.387,0.810,0.65278,17.32,43.00,9.030227,4.177632,9.863294,9.999747,5.037221,8.681579
GNQ,GNQ,Equatorial Guinea,0.650,NaN,NaN,1.70,50.20,5.717884,NaN,NaN,9.999980,6.823821,8.495596
SOM,SOM,Somalia,0.380,0.769,0.22928,32.73,36.80,9.118388,4.851974,9.953603,9.999517,3.498759,8.444758
TCD,TCD,Chad,0.394,0.776,1.39343,14.07,37.40,8.942065,4.736842,9.705355,9.999796,3.647643,8.309913
AFG,AFG,Afghanistan,0.462,0.622,0.17753,119.74,29.40,8.085642,7.269737,9.964638,9.998218,1.662531,8.292326
CXR,AUS,Christmas Island,0.310,NaN,NaN,60890.77,34.33,10.000000,NaN,NaN,9.090841,2.885856,8.170463
DMA,DMA,Dominica,0.740,NaN,3.50866,6.42,44.40,4.584383,NaN,9.254297,9.999910,5.384615,8.136342
